In [1]:
# Dynamic Pricing for Urban Parking Lots

# This notebook covers:
# 1. Data Exploration
# 2. Model 1: Baseline Linear Model
# 3. Model 2: Demand-Based Pricing
# 4. Model 3: Competitive Pricing
# 5. Real-Time Simulation using Pathway
# 6. Bokeh Visualization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt


In [2]:
# Load the dataset
df = pd.read_csv("dataset.csv")

In [3]:
# Preprocessing
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
vehicle_type_map = {'bike': 1, 'car': 2, 'truck': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_map)
df['Datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

In [4]:
# Step 1: Baseline Model

def baseline_price(df, base_price=10.0, alpha=2.0):
    df = df.copy()
    df['BaselinePrice'] = base_price + alpha * (df['Occupancy'] / df['Capacity'])
    return df


In [5]:
# Step 2: Demand-Based Model

def demand_price(df, base_price=10.0, alpha=1.5, beta=0.5, gamma=0.3, delta=2.0, epsilon=0.5, lam=0.2):
    df = df.copy()
    demand = (
        alpha * (df['Occupancy'] / df['Capacity']) +
        beta * df['QueueLength'] -
        gamma * df['TrafficLevel'] +
        delta * df['IsSpecialDay'] +
        epsilon * df['VehicleTypeWeight']
    )
    norm_demand = (demand - demand.min()) / (demand.max() - demand.min())
    df['DemandPrice'] = base_price * (1 + lam * norm_demand)
    df['DemandPrice'] = df['DemandPrice'].clip(lower=5, upper=20)
    return df

In [ ]:
# Step 3: Competitive Pricing

def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6371 * c
    return km

In [7]:
# Optimized competitive pricing using precomputed locations by SystemCodeNumber

def competitive_price(df, radius_km=1.0):
    df = df.copy()
    lot_centers = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first().reset_index()
    lot_prices = df.groupby('SystemCodeNumber')['DemandPrice'].mean().reset_index()
    merged = pd.merge(lot_centers, lot_prices, on='SystemCodeNumber')

    price_map = {}
    for _, row in merged.iterrows():
        nearby = merged[merged['SystemCodeNumber'] != row['SystemCodeNumber']].copy()
        nearby['dist'] = nearby.apply(lambda r: haversine(row['Latitude'], row['Longitude'], r['Latitude'], r['Longitude']), axis=1)
        nearby_prices = nearby[nearby['dist'] <= radius_km]['DemandPrice']
        if not nearby_prices.empty:
            price_map[row['SystemCodeNumber']] = (row['DemandPrice'] + nearby_prices.mean()) / 2
        else:
            price_map[row['SystemCodeNumber']] = row['DemandPrice']

    df['CompetitivePrice'] = df['SystemCodeNumber'].map(price_map)
    return df


In [8]:

# Apply models
df1 = baseline_price(df)
df2 = demand_price(df1)
df3 = competitive_price(df2)

In [9]:
# Visualization
import bokeh.plotting as bp
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
output_notebook()

plot_data = df3[df3['SystemCodeNumber'] == df3['SystemCodeNumber'].iloc[0]]
plot_data = plot_data.sort_values('Datetime')

source = ColumnDataSource(data={
    'time': plot_data['Datetime'],
    'baseline': plot_data['BaselinePrice'],
    'demand': plot_data['DemandPrice'],
    'competitive': plot_data['CompetitivePrice']
})

p = bp.figure(x_axis_type="datetime", title="Parking Lot Pricing Over Time", width=900)
p.line('time', 'baseline', source=source, legend_label="Baseline", line_width=2, color='blue')
p.line('time', 'demand', source=source, legend_label="Demand-Based", line_width=2, color='green')
p.line('time', 'competitive', source=source, legend_label="Competitive", line_width=2, color='red')

p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price"

show(column(p))


Loading BokehJS ...

In [10]:
# ---------------------------------------------
# 🔁 Real-Time Simulation with Pathway (Mocked)
# ---------------------------------------------

# Simulate real-time data chunking in 30-min steps
import time
unique_times = sorted(df3['Datetime'].unique())

print("\n\n--- Starting Simulated Stream ---\n")

for t in unique_times[:10]:  # simulate first 10 steps only for demo
    current_data = df3[df3['Datetime'] == t]
    print(f"\nTime: {t}")
    for _, row in current_data.iterrows():
        print(f"Lot: {row['SystemCodeNumber']}, Base: {row['BaselinePrice']:.2f}, Demand: {row['DemandPrice']:.2f}, Competitive: {row['CompetitivePrice']:.2f}")
    time.sleep(1)  # simulate delay

print("\n--- Simulation Complete ---")




--- Starting Simulated Stream ---


Time: 2016-10-04 07:59:00
Lot: BHMBCCMKT01, Base: 10.21, Demand: 10.21, Competitive: 10.58
Lot: BHMBCCTHL01, Base: 10.62, Demand: 10.36, Competitive: 10.67
Lot: BHMEURBRD01, Base: 10.50, Demand: 10.34, Competitive: 10.63
Lot: BHMMBMMBX01, Base: 10.77, Demand: 10.38, Competitive: 10.69
Lot: BHMNCPHST01, Base: 10.39, Demand: 10.24, Competitive: 10.64
Lot: BHMNCPNST01, Base: 11.03, Demand: 10.42, Competitive: 10.64
Lot: Broad Street, Base: 10.52, Demand: 10.35, Competitive: 10.62
Lot: Others-CCCPS105a, Base: 10.71, Demand: 10.37, Competitive: 10.68
Lot: Others-CCCPS119a, Base: 10.14, Demand: 10.20, Competitive: 10.61
Lot: Others-CCCPS135a, Base: 10.56, Demand: 10.35, Competitive: 10.71
Lot: Others-CCCPS202, Base: 10.37, Demand: 10.23, Competitive: 10.65
Lot: Others-CCCPS8, Base: 10.67, Demand: nan, Competitive: 10.66
Lot: Others-CCCPS98, Base: 10.38, Demand: nan, Competitive: 10.65
Lot: Shopping, Base: 10.64, Demand: nan, Competitive: 10.68

Time: 201